#### This notebook will guide you through the code behind vits(https://github.com/jaywalnut310/vits),a classical e2e TTS model.

#### Import required package

In [2]:
import os
import json
import argparse
import itertools
import math
import logging
import json
import subprocess
import numpy as np
from scipy.io.wavfile import read
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import random
import librosa
import librosa.util as librosa_util
from librosa.util import normalize, pad_center, tiny
from scipy.signal import get_window
from scipy.io.wavfile import read
from librosa.filters import mel as librosa_mel_fn

c:\Users\TokaiTeio\.conda\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\TokaiTeio\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\TokaiTeio\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\TokaiTeio\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### In a e2e TTS system, the first thing is to understand the training data.
##### the training data of VITS consists of two ingredients, text and coresponding audio.

VITS reads data through a txt. Inside the txt, the data is seened as below.

DUMMY1/LJ050-0234.wav|It has used...

DUMMY1/LJ019-0373.wav|to avail himself...